In [ ]:
#!pip install networkx

import networkx as nx
import numpy as np
import random
import requests
import time
import math
import os
from collections import deque, Counter



def import_map_as_matrix(file_path):
    """
    Importuje 2D mapu ako maticu.
    
    subor musi obsahovat maticu so spravnym syntaxom
    - 0 je prazdne miesto
    - 1 je start position
    - 2 je prekazka
    - 3 je goal position
    """
    try:
        with open(file_path, 'r') as file:
            content = file.read()
            
        # eval function to parse the matrix representation
        matrix = eval(content)
        
        # validate: find start and goal
        start_pos = None
        goal_pos = None
        
        for i in range(len(matrix)):
            for j in range(len(matrix[i])):
                if matrix[i][j] == 1:
                    start_pos = (i, j)
                elif matrix[i][j] == 3:
                    goal_pos = (i, j)
        
        print(f"Map loaded: {len(matrix)}x{len(matrix[0])}")
        print(f"Start position: {start_pos}")
        print(f"Goal position: {goal_pos}")
        
        return matrix, start_pos, goal_pos
    
    except Exception as e:
        print(f"Error loading map: {e}")
        return None, None, None

# ====================================================================== 
# greedy best first search and support functions
# ====================================================================== 
def heuristic(matrix, node, goal):
    """
    Vypocita manhattan distance pre kazdy node a vrati maticu s vzialenostami.

    je to heuristicka funkcia pre greedy best first search
    - preiteruje cez vsetky node a vypocita manhattan distance
    abs(x1 - x2) + abs(y1 - y2)
    """
    goalArray = np.array(goal) # tuple na array
    distance = 0
    matrix_of_distance = matrix
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            node = (i, j)
            nodeArray = np.array(node)
            if matrix[i][j] == 2: # prekazka
                manhattan_distance = 9999 # infinity pre prekazky
            else:
                manhattan_distance = int(np.abs(nodeArray[0] - goalArray[0]) + np.abs(nodeArray[1] - goalArray[1])) # int inac to vrati np.int64(12)
            matrix_of_distance[i][j] = manhattan_distance
            

    #print(f"Manhattan distance from {node} to {goal}: {manhattan_distance}")
    return matrix_of_distance

def greedy_best_first_search(mapa, start, goal):
    """
    Greedy Best First Search
    heurisitka je manhattan distance
    skuma susedne body vybera mensie cislo a ide tam dokym nedosiahne 0
    """
    distance_matrix = heuristic(mapa, start, goal)

    aktualny_riadok, aktualny_stlpec = start
    path = [(aktualny_riadok, aktualny_stlpec)]  # ulozenie path

    aktualna_hodnota = distance_matrix[aktualny_riadok][aktualny_stlpec]

    while aktualna_hodnota != 0:
        susedia = []

        """
        skumanie susednych bodov:

        - pamataj bod v ktorom sme

        pozri vlavo, ak je mimo skip
        pozri vpravo, ak je mimo skip
        pozri hore, ak je mimo skip
        pozri dole, ak je mimo skip

        vyhodnot ktory je najmensi z nich 
        ak su dva rovnake vyber nahodne
        pridaj tento bod do navstivenych bodov
        nastav vybrany bod ako ten bod v ktorom sme a opakuj
        """

        # uloz pozicie s hodnotou heuristiky
        if aktualny_riadok > 0:
            susedia.append(((aktualny_riadok - 1, aktualny_stlpec), distance_matrix[aktualny_riadok - 1][aktualny_stlpec]))
        if aktualny_riadok < len(distance_matrix) - 1:
            susedia.append(((aktualny_riadok + 1, aktualny_stlpec), distance_matrix[aktualny_riadok + 1][aktualny_stlpec]))
        if aktualny_stlpec > 0:
            susedia.append(((aktualny_riadok, aktualny_stlpec - 1), distance_matrix[aktualny_riadok][aktualny_stlpec - 1]))
        if aktualny_stlpec < len(distance_matrix[0]) - 1:
            susedia.append(((aktualny_riadok, aktualny_stlpec + 1), distance_matrix[aktualny_riadok][aktualny_stlpec + 1]))

        #print("Susedia", susedia)

        # vyber minimum podľa heuristiky
        kandidati = []
        terajsia_pozicia = (aktualny_riadok, aktualny_stlpec)
        #Susedia [((1, 4), 7), ((3, 4), 5), ((2, 3), 7), ((2, 5), 5)]
        for pozicia, hodnota in susedia:
            if hodnota < aktualna_hodnota:
                kandidati.append(pozicia)

        # náhodne vyber, ak viacero rovnakých
        vybrany_bod = random.choice(kandidati)

        aktualny_riadok, aktualny_stlpec = vybrany_bod
        path.append(vybrany_bod)
        aktualna_hodnota = distance_matrix[aktualny_riadok][aktualny_stlpec]

    return path

    

def path_planner(self, start, goal):
    # budeme to robit v grafe
    # berie dvojicu suradnic start a dvojicu suradnic goal
    # v mape block.py vrati dve premenne:
    # 1. frontu resp array. obsahujucu suvislu postupnost dvojic suradnic zo statu do ciela
    # 2. pole expandovanych uzlov k vyfarbeniu
    # implementovat: greedy best first search, dijkstra a a*

    # =================================================================
    # 1. greedy best first search
    # =================================================================
    # co to robi
    # skuma cestu bez ohladu ci je to naozaj najlepsia cesta
    # skuma body a vyberie cestu co ma "najlepsi cenu"
    # opakuje sa dokym sa nedostane do ciela
    greedy_path = greedy_best_first_search(self.map, start, goal)

    # =================================================================
    # 2. dijkstra
    # =================================================================
    # co to robi
    # 
    
    # =================================================================
    # 3. a*
    # =================================================================
    return 0

if __name__ == "__main__":
    map, start, goal = import_map_as_matrix('2Dmapa.txt')
    print(greedy_best_first_search(map, start, goal))

Map loaded: 7x7
Start position: (0, 0)
Goal position: (6, 6)
[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (5, 1), (5, 2), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6)]
